<a href="https://colab.research.google.com/github/haizzzi/lecture_notes/blob/main/Llama_3_8b_base_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0) 런타임: GPU(T4) 선택
# 1) 필수 패키지 설치
!pip -q install -U transformers accelerate bitsandbytes sentencepiece



In [ ]:
# 2) (한번만) 허깅페이스 로그인
from huggingface_hub import login
login("hf_xxx...")  # 이러면 위젯 출력이 안 남음


In [ ]:
import os
os.environ["HF_HOME"] = "/content/drive/MyDrive/huggingface_cache"  # <- 여기로 모델/토크나이저 캐시 저장

# 3) 모델 로드 (4-bit)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_id = "meta-llama/Llama-3.1-8B"   # ✅ 정확한 ID (Base)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",       # nf4 권장
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 필요시 token="hf_..." 인자로 명시 가능 (전역 login 안 했을 때)
tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)  # , token="hf_..."
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,      # bf16 권장
)



In [ ]:
# 4) 간단 생성
prompt = "Geoffrey Everest Hinton (born 6 December 1947) is a British-Canadian computer scientist, cognitive scientist, and cognitive psychologist known for his work on artificial neural networks, which earned him the title the Godfather of AI.[10]"
inputs = tok(prompt, return_tensors="pt").to(model.device)
with torch.inference_mode():
    out = model.generate(
        **inputs,
        max_new_tokens=500,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tok.eos_token_id
    )
print(tok.decode(out[0], skip_special_tokens=True))
